In [ ]:
import os
import sys
import pickle

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')

from conformal_prediction.conformal_risk_uncertaint_prediction import ConformalRiskUncertaintyPrediction
from load_data.load_conformal_data import load_results
from conformance_checking.conformance import ConformanceChecking

In [ ]:
path_to_be_model = '../../../../process_models/Helpdesk.bpmn'

path_d_inference_results = '../../../../../../data/Helpdesk/gn_l4'

In [ ]:
results = load_results(path_d_con_results=path_d_inference_results)

In [ ]:
cc = ConformanceChecking(path_process_model=path_to_be_model)

In [ ]:
# Load results form Conformal Analysis .pkl file
output_dir = '../../../../../../data/Helpdesk/conformal_val/conformal_conformance_results'

filename = os.path.join(output_dir, 'results.pkl')
with open(filename, 'rb') as f:
    loaded_results = pickle.load(f)
print(f"Loaded data from {filename}")

# Data loaded:
# float value
alpha = loaded_results["alpha"]
# dict: prefix length, dict: case name, (List of fitness scores, mean fitness scores, std fitness scores)
case_name_fitness_scores_per_prefix_length = loaded_results["case_name_fitness_scores_per_prefix_length"]
# dict: prefix length, (mean target fitness score, mean most likely fitness score, mean means samples fitness scores)
mean_tgts_ml_samples_per_prefix_length = loaded_results["mean_tgts_ml_samples_per_prefix_length"]
# dict: prefix length, (mean of std within fitness scores, std of mean sampled fitness scores)
std_samples_per_prefix_length = loaded_results["std_samples_per_prefix_length"]
# dict: prefix length, r: sampled fitness score threshold lower bound with 1-alpha coverage.
q_samples_per_prefix_length = loaded_results["q_samples_per_prefix_length"]

In [ ]:
# Get Dict with prefix length and threshold values
mean_fitness_prefix_lens = {}
r_fitness_prefix_lens = {}

for prefix_length in mean_tgts_ml_samples_per_prefix_length.keys():
    _, _, mean_fitness_samples = mean_tgts_ml_samples_per_prefix_length[prefix_length]
    r_fitness_samples = q_samples_per_prefix_length[prefix_length]
    
    mean_fitness_prefix_lens[prefix_length] = mean_fitness_samples
    r_fitness_prefix_lens[prefix_length] = r_fitness_samples

# print(mean_fitness_prefix_lens)
# print(r_fitness_prefix_lens)

In [ ]:
conf_risk_unc_pred = ConformalRiskUncertaintyPrediction(d_inference_results=results,
                                                        conformance_object=cc,
                                                        log_name='Helpdesk')

high_risk_group, risk_group = conf_risk_unc_pred.risk_set_prediction(mean_conformal_fitness_prefix_lengths=mean_fitness_prefix_lens,
                                                                     r_conformal_fitness_prefix_lengths=r_fitness_prefix_lens)

# print(high_risk_group)
# print(risk_group)

In [ ]:
# Save results to .pkl in conformal_val directory
output_dir   = '../../../../../../data/Helpdesk/conformal_prediction_gn_l4'

os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

filename = os.path.join(output_dir, f'risk_inference_set.pkl')
with open(filename, 'wb') as f:
    pickle.dump(risk_group, f)
print(f"Saved {len(results)} entries to {filename}")

filename = os.path.join(output_dir, f'high_risk_inference_set.pkl')
with open(filename, 'wb') as f:
    pickle.dump(high_risk_group, f)
print(f"Saved {len(results)} entries to {filename}")

In [ ]:
filename = os.path.join(output_dir, 'risk_inference_set.pkl')
with open(filename, 'rb') as f:
    loaded_results_risk = pickle.load(f)
print(f"Loaded data from {filename}")

filename = os.path.join(output_dir, 'high_risk_inference_set.pkl')
with open(filename, 'rb') as f:
    loaded_results_high_risk = pickle.load(f)
print(f"Loaded data from {filename}")